In [1]:
#include<mlpack/xeus-cling.hpp>
#include <mlpack/core.hpp>

#include <mlpack/methods/ann/init_rules/gaussian_init.hpp>
#include <mlpack/methods/ann/loss_functions/sigmoid_cross_entropy_error.hpp>
#include <mlpack/methods/ann/gan/gan.hpp>
#include <mlpack/methods/ann/layer/layer.hpp>
#include <mlpack/methods/softmax_regression/softmax_regression.hpp>
#include<mlpack/methods/ann/ffn.hpp>

#include <ensmallen.hpp>

// #include "catch.hpp"
// #include "test_catch_tools.hpp"
// #include "serialization.hpp"

// using namespace mlpack;
// using namespace mlpack::ann;
// using namespace mlpack::math;
// using namespace mlpack::regression;
// using namespace std::placeholders;


In [2]:
using namespace mlpack::ann;

In [3]:
using namespace mlpack;

In [4]:
using namespace mlpack::math;

In [5]:
using namespace mlpack::regression;

In [6]:
using namespace std::placeholders;

In [7]:
  size_t dNumKernels = 32;
  size_t discriminatorPreTrain = 5;
  size_t batchSize = 5;
  size_t noiseDim = 100;
  size_t generatorUpdateStep = 1;
  size_t numSamples = 10;
  double stepSize = 0.0003;
  double eps = 1e-8;
  size_t numEpoches = 1;
  double tolerance = 1e-5;
  int datasetMaxCols = 10;
  bool shuffle = true;
  double multiplier = 10;

In [8]:

  std::cout << std::boolalpha
      << " batchSize = " << batchSize << std::endl
      << " generatorUpdateStep = " << generatorUpdateStep << std::endl
      << " noiseDim = " << noiseDim << std::endl
      << " numSamples = " << numSamples << std::endl
      << " stepSize = " << stepSize << std::endl
      << " numEpoches = " << numEpoches << std::endl
      << " tolerance = " << tolerance << std::endl
      << " shuffle = " << shuffle << std::endl;


 batchSize = 5
 generatorUpdateStep = 1
 noiseDim = 100
 numSamples = 10
 stepSize = 0.0003
 numEpoches = 1
 tolerance = 1e-05
 shuffle = true


In [ ]:
  arma::mat trainData;
  trainData.load("mnist_first250_training_4s_and_9s.arm");
  std::cout << arma::size(trainData) << std::endl;

  trainData = trainData.cols(0, datasetMaxCols - 1);

  size_t numIterations = trainData.n_cols * numEpoches;
  numIterations /= batchSize;

  std::cout << "Dataset loaded (" << trainData.n_rows << ", "
            << trainData.n_cols << ")" << std::endl;
  std::cout << trainData.n_rows << "--------" << trainData.n_cols << std::endl;

  // Create the Discriminator network.
  FFN<mlpack::ann::SigmoidCrossEntropyError<> > discriminator;
  discriminator.Add<Convolution<> >(1, dNumKernels, 5, 5, 1, 1, 2, 2, 28, 28);
  discriminator.Add<ReLULayer<> >();
  discriminator.Add<MeanPooling<> >(2, 2, 2, 2);
  discriminator.Add<Convolution<> >(dNumKernels, 2 * dNumKernels, 5, 5, 1, 1,
      2, 2, 14, 14);
  discriminator.Add<ReLULayer<> >();
  discriminator.Add<MeanPooling<> >(2, 2, 2, 2);
  discriminator.Add<Linear<> >(7 * 7 * 2 * dNumKernels, 1024);
  discriminator.Add<ReLULayer<> >();
  discriminator.Add<Linear<> >(1024, 1);

  // Create the Generator network.
  FFN<mlpack::ann::SigmoidCrossEntropyError<> > generator;
  generator.Add<Linear<> >(noiseDim, 3136);
  generator.Add<BatchNorm<> >(3136);
  generator.Add<ReLULayer<> >();
  generator.Add<Convolution<> >(1, noiseDim / 2, 3, 3, 2, 2, 1, 1, 56, 56);
  generator.Add<BatchNorm<> >(39200);
  generator.Add<ReLULayer<> >();
  generator.Add<BilinearInterpolation<> >(28, 28, 56, 56, noiseDim / 2);
  generator.Add<Convolution<> >(noiseDim / 2, noiseDim / 4, 3, 3, 2, 2, 1, 1,
      56, 56);
  generator.Add<BatchNorm<> >(19600);
  generator.Add<ReLULayer<> >();
  generator.Add<BilinearInterpolation<> >(28, 28, 56, 56, noiseDim / 4);
  generator.Add<Convolution<> >(noiseDim / 4, 1, 3, 3, 2, 2, 1, 1, 56, 56);
  generator.Add<TanHLayer<> >();

  // Create GAN.
  GaussianInitialization gaussian(0, 1);
  ens::Adam optimizer(stepSize, batchSize, 0.9, 0.999, eps, numIterations,
      tolerance, shuffle);
  std::function<double()> noiseFunction = [] () {
      return math::RandNormal(0, 1);};
  GAN<FFN<mlpack::ann::SigmoidCrossEntropyError<> >, GaussianInitialization,
      std::function<double()> > gan(generator, discriminator,
      gaussian, noiseFunction, noiseDim, batchSize, generatorUpdateStep,
      discriminatorPreTrain, multiplier);

  std::cout << "Training..." << std::endl;
  std::stringstream stream;
  double objVal = gan.Train(trainData, optimizer, ens::ProgressBar(70, stream));
//   REQUIRE(stream.str().length() > 0);
//   REQUIRE(std::isfinite(objVal) == true);

//   // Generate samples.
//   std::cout << "Sampling..." << std::endl;
//   arma::mat noise(noiseDim, batchSize);
//   size_t dim = std::sqrt(trainData.n_rows);
//   arma::mat generatedData(2 * dim, dim * numSamples);

//   for (size_t i = 0; i < numSamples; ++i)
//   {
//     arma::mat samples;
//     noise.imbue( [&]() { return noiseFunction(); } );

//     gan.Generator().Forward(noise, samples);
//     samples.reshape(dim, dim);
//     samples = samples.t();

//     generatedData.submat(0, i * dim, dim - 1, i * dim + dim - 1) = samples;

//     samples = trainData.col(math::RandInt(0, trainData.n_cols));
//     samples.reshape(dim, dim);
//     samples = samples.t();

//     generatedData.submat(dim,
//         i * dim, 2 * dim - 1, i * dim + dim - 1) = samples;
//   }

//   std::cout << "Output generated!" << std::endl;

//   // Check that Serialization is working correctly.
//   arma::mat orgPredictions;
//   gan.Predict(noise, orgPredictions);

//   GAN<FFN<mlpack::ann::SigmoidCrossEntropyError<> >, GaussianInitialization,
//       std::function<double()> > ganText(generator, discriminator,
//       gaussian, noiseFunction, noiseDim, batchSize, generatorUpdateStep,
//       discriminatorPreTrain, multiplier);

//   GAN<FFN<mlpack::ann::SigmoidCrossEntropyError<> >, GaussianInitialization,
//       std::function<double()> > ganXml(generator, discriminator,
//       gaussian, noiseFunction, noiseDim, batchSize, generatorUpdateStep,
//       discriminatorPreTrain, multiplier);

//   GAN<FFN<mlpack::ann::SigmoidCrossEntropyError<> >, GaussianInitialization,
//       std::function<double()> > ganBinary(generator, discriminator,
//       gaussian, noiseFunction, noiseDim, batchSize, generatorUpdateStep,
//       discriminatorPreTrain, multiplier);

//   SerializeObjectAll(gan, ganXml, ganText, ganBinary);

//   arma::mat predictions, xmlPredictions, textPredictions, binaryPredictions;
//   gan.Predict(noise, predictions);
//   ganXml.Predict(noise, xmlPredictions);
//   ganText.Predict(noise, textPredictions);
//   ganBinary.Predict(noise, binaryPredictions);

//   CheckMatrices(orgPredictions, predictions);
//   CheckMatrices(orgPredictions, xmlPredictions);
//   CheckMatrices(orgPredictions, textPredictions);
//   CheckMatrices(orgPredictions, binaryPredictions);

In file included from input_line_5:1:
In file included from /home/viole/anaconda3/envs/notebook/include/xeus/xinterpreter.hpp:13:
In file included from /home/viole/anaconda3/envs/notebook/bin/../lib/gcc/x86_64-conda-linux-gnu/7.5.0/../../../../x86_64-conda-linux-gnu/include/c++/7.5.0/functional:58:
/home/viole/anaconda3/envs/notebook/bin/../lib/gcc/x86_64-conda-linux-gnu/7.5.0/../../../../x86_64-conda-linux-gnu/include/c++/7.5.0/bits/std_function.h:264:34: error: no matching constructor for initialization of '(lambda at input_line_16:43:43)'
        { ::new (__functor._M_access()) _Functor(std::move(__f)); }
                                        ^        ~~~~~~~~~~~~~~
/home/viole/anaconda3/envs/notebook/bin/../lib/gcc/x86_64-conda-linux-gnu/7.5.0/../../../../x86_64-conda-linux-gnu/include/c++/7.5.0/bits/std_function.h:239:4: note: in instantiation of member function 'std::_Function_base::_Base_manager<(lambda at
      input_line_16:43:43)>::_M_init_functor' requested here
        { 